In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

* 이전 step까지 노드 분류 작업을 위해 전체 배치 방식으로만 그래프 신경망을 훈련했습니다. 
    * 특히, 이는 모든 노드의 은닉 표현이 병렬로 계산되었고 다음 계층에서 재사용할 수 있음을 의미합니다.
    * 그러나 큰 그래프에서 작업을 수행하고자 하면 메모리 소비가 폭발적으로 증가하기 때문에 이 계획은 더 이상 실현 가능하지 않습니다. 
    * 예를 들어 약 천만 개의 노드와 128개의 hidden feature 차원이 있는 그래프는 이미 각 계층에 대해 약 5GB의 GPU 메모리를 소비합니다.
* 따라서, 최근 그래프 신경망을 더 큰 그래프로 확장하려는 노력이 있다.
    * 이러한 접근법 중 하나는 Cluster-GCN(Chiang et al. (2019)으로 알려져 있으며, 이는 그래프를 미니 배치 방식으로 운영할 수 있는 하위 그래프(sub-graph)로 사전 파티셔닝(pre partitioning)하는 것을 기반으로 한다

In [1]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('==================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===============================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...
Done!

Dataset: PubMed():
Number of graphs: 1
Number of features: 500
Number of classes: 3

Data(edge_index=[2, 88648], test_mask=[19717], train_mask=[19717], val_mask=[19717], x=[19717, 500], y=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.003
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


* node
    * 19717 n
    * 이 수의 노드는 GPU 메모리에 쉽게 맞아야 하지만 그럼에도 불구하고 PyTorch Geometric 내에서 GNN을 확장할 수 있는 방법을 보여주는 좋은 예입니다.
* Cluster-GCN(Chiang et al. (2019)은 그래프 분할 알고리즘을 기반으로 그래프를 먼저 하위 그래프로 분할하여 작동합니다.

따라서 GNN은 특정 하위 그래프 내에서만 교란하도록 제한되며, 이는 이웃의 폭발 문제를 생략합니다.

그러나 그래프를 분할한 후 일부 링크가 제거되어 편향된 추정으로 인해 모델의 성능이 제한될 수 있습니다. 

또한 이 문제를 해결하기 위해 Cluster-GCN은 미니 배치 내에 클러스터 간 링크를 통합하고, 다음과 같은 확률적 분할 체계를 구축합니다.

In [2]:
from torch_geometric.data import ClusterData, ClusterLoader

torch.manual_seed(12345)
cluster_data = ClusterData(data, num_parts=128)  # 1. Create subgraphs.
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)  # 2. Stochastic partioning scheme.

print()
total_num_nodes = 0
for step, sub_data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of nodes in the current batch: {sub_data.num_nodes}')
    print(sub_data)
    print()
    total_num_nodes += sub_data.num_nodes

print(f'Iterated over {total_num_nodes} of {data.num_nodes} nodes!')

Computing METIS partitioning...
Done!

Step 1:
Number of nodes in the current batch: 4946
Data(edge_index=[2, 15230], test_mask=[4946], train_mask=[4946], val_mask=[4946], x=[4946, 500], y=[4946])

Step 2:
Number of nodes in the current batch: 4916
Data(edge_index=[2, 18420], test_mask=[4916], train_mask=[4916], val_mask=[4916], x=[4916, 500], y=[4916])

Step 3:
Number of nodes in the current batch: 4926
Data(edge_index=[2, 16202], test_mask=[4926], train_mask=[4926], val_mask=[4926], x=[4926, 500], y=[4926])

Step 4:
Number of nodes in the current batch: 4929
Data(edge_index=[2, 17260], test_mask=[4929], train_mask=[4929], val_mask=[4929], x=[4929, 500], y=[4929])

Iterated over 19717 of 19717 nodes!


In [3]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)


In [5]:
import sys
class Printer():
    """Print things to stdout on one line dynamically"""
    def __init__(self,num_period=10):
        self.num_period = num_period
        self.init_value = 0

    def __call__(self,data) :
        if self.init_value % self.num_period == 0 :
            print('\n'+data.__str__())
            self.init_value = 1
        else :
            sys.stdout.write("\r\x1b[K"+data.__str__())
            sys.stdout.flush()
            self.init_value += 1 

printf = Printer(num_period=50)

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()

      for sub_data in train_loader:  # Iterate over each mini-batch.
          out = model(sub_data.x, sub_data.edge_index)  # Perform a single forward pass.
          loss = criterion(out[sub_data.train_mask], sub_data.y[sub_data.train_mask])  # Compute the loss solely based on the training nodes.
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          optimizer.zero_grad()  # Clear gradients.

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      
      accs = []
      for mask in [data.train_mask, data.val_mask, data.test_mask]:
          correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
          accs.append(int(correct.sum()) / int(mask.sum()))  # Derive ratio of correct predictions.
      return accs

for epoch in range(1, 51):
    loss = train()
    train_acc, val_acc, test_acc = test()
    printf(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')


Epoch: 001, Train: 0.3333, Val Acc: 0.4160, Test Acc: 0.4070
Epoch: 050, Train: 0.9833, Val Acc: 0.7980, Test Acc: 0.7810

In [6]:
test()

[0.9833333333333333, 0.798, 0.781]

# [gnn examples](https://github.com/rusty1s/pytorch_geometric/tree/master/examples)